
# How much of the web runs on renewable power?

I'm trying to work this out as some research for the Planet Friendly Web Guide, as I can't find much info online myself.

Here's my current thinking on how you might do this. There are obvious holes in this, but it's a start, and having _something_ publicly accessible presumably better than what we have now, this is _nowt_.


## My current thinking about how to do this

- Take a dataset that is representative of the entire web, listing the most popular sites
- Find a away to tell if these sites are running on renewable power
- Run this check on the sites
- Share results

I'll explain these in more detail.


### Find a dataset that's representative

Right now, the de-factor dataset used to represent how the web is used, at least for web performance optimisation and marketing is Alexa's list of the top million sites, as described on [the HTTP Archive site][1].

[1]: https://httparchive.org/about.php#faq

Iniitally I thought you might need to come up with some on Big Query er, query, like described here on [Ilya Grigorik's website](https://www.igvita.com/2013/06/20/http-archive-bigquery-web-performance-answers/). This saves needing to run our own infra, and should make reproducible.

But given we only need the url, and rank for now we might be able to do this without needing to learn big query, or how to make API calls when running a query on the BigQuery.

The list of the top million is downloadable as a compressed file, that's about 9mb compressed, and 23mb uncompressed. This is downloadable from an Amazon S3 bucket, at the link below

http://s3.amazonaws.com/alexa-static/top-1m.csv.zip

### Find a way to tell if it's running on renewable power

This in itself is a hellishly complex topic - as there loads of ways to declare that your infrastructure is running on renewable power, from _actually using renewables directly_ to using various financial instruments to buy the same about renewable power you're really using in a non-renewably powered datacentre, to another form of offsets.

But for now, to start with, let's try using the [Green Web API](https://www.thegreenwebfoundation.org/green-web-feed/) - the API is simple to understand, and in use.

This API checks a domain against who is hosting it, and checks if the hosting company has declared they're running on renewables. It's not clear to me yet what specific checks there, and _how_ they do this, but again, it's our first pass at this.

### Run this check

It's not technically complex to loop through a csv file, and for each row hit an API. In python the pseudocode might look like this:

```python

green_or_not_list = []

for row in open('path/to/dataset.csv'):
    # do some stuff to find the right column, or tell python we're 
    # looking at a CSV file
    site_url = pull_url_column_from(row)
    
    result = check_against_green_web_api(site_url)
    green_or_not_list.push(result)

```

Thing is this, this involves hitting the API a million times if there are a million rows in the dataset.

It's in the very least polite to contact the nice folks at Green Web before we hit their API a million times - one for each url in the Alexa dataset though.

The nice thing in the dataset from Alex is that the csv file as two columns, domain and rank, we can do this with the first 10, or maybe 100 results, and get something halfway interesting back, without hosing somebody else's API.

### Share Results

We should really to make this reproducible.

It looks like packaging this up as a [binder][] would be a good step for this. I haven't made one before, but it's worth a try, right?

[binder]: https://mybinder.org/

------------------------------------------
